In [201]:
import blurbs

import numpy as np
import matplotlib.pyplot as plt
import csv
from functools import reduce
import re 
from collections import defaultdict

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import pandas as pd

import math



# imports from my modules
import imp
import feature_generation
imp.reload(feature_generation)
import bullets
imp.reload(bullets)

# yay neural nets
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.activations import linear, relu, sigmoid


In [195]:
DIR = '/home/bryce/Projects/Data_Science/Apt_Prices/csvs/march8/'

address_file =  DIR + 'pd_address_info.csv' 
apt_file = DIR + 'pd_apt_info.csv'

addr_df = pd.read_csv(address_file, sep=';')
apt_df = pd.read_csv(apt_file, sep=';')

((apt_train, addr_train), (apt_val, addr_val), (apt_test, addr_test)) = feature_generation.split_data(apt_df=apt_df, addr_df=addr_df)

print("addr_train shape: ", addr_train.shape)
print("addr_val shape: ", addr_val.shape)
print("addr_test shape: ", addr_test.shape)

addr_val.tail()

apt_df shape before:  (3166, 8)
addr_df shape before:  (433, 7)
apt_df shape after:  (3151, 8)
addr_df shape after:  (321, 7)
addr_train shape:  (208, 7)
addr_val shape:  (64, 7)
addr_test shape:  (49, 7)


/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addr_df.drop_duplicates(subset=['address'], inplace=True)


,address,url,title,bullets,blurb,latlng,latlng_len
59,"1001 Riverland Woods Pl, Charleston, SC 29412",https://www.apartments.com/riverland-woods-ch...,\tRiverland Woods - 1001 Riverland Woods Pl C...,()Maintenance on site. ()Property Manager on S...,LIVE LUX. Minutes from Folly Beach and Downto...,"(32.71656411329172, -79.97034047647085)",39
60,"4251 S Rhett Ave, North Charleston, SC 29405",https://www.apartments.com/park-circle-villag...,\tPark Circle Village - 4251 S Rhett Ave Nort...,()Maintenance on site. ()Property Manager on S...,Park Circle Village is an apartment community...,"(32.872409354923604, -79.984937)",2
61,"1011 Diana St, Charleston, SC 29407",https://www.apartments.com/1011-diana-st-char...,"1011 Diana St, Charleston, SC 29407 - House ...",()Laundry Facilities,"3 br, 2 ba 1591 sqft home in the St. Andrews ...","(32.795358, -79.99765166666667)",31
62,"7724 Brookdale Blvd, North Charleston, SC 29418",https://www.apartments.com/3-br-2-bath-house-...,"\t3 br, 2 bath House - 7724 Brookdale Blvd - ...",()Dishwasher,- - No Pets Allowed (RLNE7877909)...,"(32.91052079538065, -80.08084934666158)",2
63,"1030 Jack Primus Rd, Charleston, SC 29492",https://www.apartments.com/parker-point-charl...,\tParker Point - 1030 Jack Primus Rd Charlest...,()Wi-Fi at Pool and Clubhouse. ()Controlled Ac...,Discover exclusive apartment living at Cainho...,"(32.916777179936474, -79.88707020464769)",40


Steps before we train the neural net

1) generate train/val/test data and inspect for sanity (DONE)
2) generate training and validation features, and inspect for sanity

In [3]:
print("addr train shape ", addr_train.shape)
addr_train.drop_duplicates(subset=['address'], inplace=True)
print("addr train shape  after dropping dups", addr_train.shape)


addr train shape  (247, 7)
addr train shape  after dropping dups (247, 7)


In [169]:
# Lets inspect these for sanity

apt_train.to_csv('/tmp/apt_train.csv', sep=';', index=False)
addr_train.to_csv('/tmp/addr_train.csv', sep=';', index=False)

addr_val.to_csv('/tmp/addr_val.csv', sep=';', index=False)
apt_val.to_csv('/tmp/apt_val.csv', sep=';', index=False)

In [202]:
feat_gen = feature_generation.FeatureGenerator()
(X_train, y_train) = feat_gen.get_training_features(apt_df_train=apt_train, addr_df_train=addr_train)
(X_val, y_val) = feat_gen.get_testing_features(apt_df_test=apt_val, addr_df_test=addr_val)
(X_test, y_test) = feat_gen.get_testing_features(apt_df_test=apt_test, addr_df_test=addr_test)

X0 shape before add lat lng  (1718, 7)
X0 shape after add lat lng  (1718, 12)
zip df shape before dropping infrequent cols  (1718, 15)
zip df shape after dropping infrequent cols  (1718, 14)
dim training blurbs, before cleaning (208,)
dim cleaned_blurbs  (208,)
training svd matrix shape  (208, 30)
about to return the training_svd_df, with shape (208, 30)
columns in dataframe:  Index(['address', 'beds', 'baths', 'sq_ft', 'location', 'zip',
       'units_in_building', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng',
       'zip_29401', 'zip_29403', 'zip_29406', 'zip_29407', 'zip_29410',
       'zip_29412', 'zip_29414', 'zip_29418', 'zip_29420', 'zip_29455',
       'zip_29464', 'zip_29466', 'zip_29485', 'zip_29492', 'latlng_len',
       'property_type', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6',
       'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'svd_12', 'svd_13',
       'svd_14', 'svd_15', 'svd_16', 'svd_17', 'svd_18', 'svd_19', 'svd_20',
       'svd_21', 'svd_22', 'svd_23', '

/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addrs_and_latlng.drop_duplicates(inplace=True)
/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addrs_and_latlng['lat'] = addrs_and_latlng.latlng.map(lambda s : self._extract_lat_and_lng(s)[0])
/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

col1:  sq_ft  col2:  zip_29492
float64
float64
col1:  units_in_building  col2:  zip_29401
int64
float64
col1:  units_in_building  col2:  zip_29403
int64
float64
col1:  units_in_building  col2:  zip_29406
int64
float64
col1:  units_in_building  col2:  zip_29407
int64
float64
col1:  units_in_building  col2:  zip_29410
int64
float64
col1:  units_in_building  col2:  zip_29412
int64
float64
col1:  units_in_building  col2:  zip_29414
int64
float64
col1:  units_in_building  col2:  zip_29418
int64
float64
col1:  units_in_building  col2:  zip_29420
int64
float64
col1:  units_in_building  col2:  zip_29455
int64
float64
col1:  units_in_building  col2:  zip_29464
int64
float64
col1:  units_in_building  col2:  zip_29466
int64
float64
col1:  units_in_building  col2:  zip_29485
int64
float64
col1:  units_in_building  col2:  zip_29492
int64
float64
We made it here!
But did we make it here?
line 1
line 2
line 3
line 4
line 5
zip df shape before dropping infrequent cols  (445, 15)
zip df shape after dro

/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addrs_and_latlng.drop_duplicates(inplace=True)
/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addrs_and_latlng['lat'] = addrs_and_latlng.latlng.map(lambda s : self._extract_lat_and_lng(s)[0])
/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [200]:
list(X_train.columns)

['address',
 'beds',
 'baths',
 'sq_ft',
 'location',
 'zip',
 'units_in_building',
 'lat',
 'lng',
 'lat^2',
 'lng^2',
 'lat * lng',
 'zip_29401',
 'zip_29403',
 'zip_29406',
 'zip_29407',
 'zip_29410',
 'zip_29412',
 'zip_29414',
 'zip_29418',
 'zip_29420',
 'zip_29455',
 'zip_29464',
 'zip_29466',
 'zip_29485',
 'zip_29492',
 'latlng_len',
 'property_type_x',
 'svd_1',
 'svd_2',
 'svd_3',
 'svd_4',
 'svd_5',
 'svd_6',
 'svd_7',
 'svd_8',
 'svd_9',
 'svd_10',
 'svd_11',
 'svd_12',
 'svd_13',
 'svd_14',
 'svd_15',
 'svd_16',
 'svd_17',
 'svd_18',
 'svd_19',
 'svd_20',
 'svd_21',
 'svd_22',
 'svd_23',
 'svd_24',
 'svd_25',
 'svd_26',
 'svd_27',
 'svd_28',
 'svd_29',
 'svd_30',
 'baths x baths',
 'baths x zip_29401',
 'baths x zip_29403',
 'baths x zip_29406',
 'baths x zip_29407',
 'baths x zip_29410',
 'baths x zip_29412',
 'baths x zip_29414',
 'baths x zip_29418',
 'baths x zip_29420',
 'baths x zip_29455',
 'baths x zip_29464',
 'baths x zip_29466',
 'baths x zip_29485',
 'baths x 

In [213]:
all_col_names = X_train.columns
cols = feature_generation.select_column_names(all_column_names=all_col_names,
                                              latlng = True,
                                              units_in_building = True,
                                              zip_feats = True,
                                              bullet_feats = True,
                                              building_type = True,
                                              blurb_feats = False,
                                              cross_term_res=[])
print(cols)
X_train_feats = X_train[cols]

['beds', 'baths', 'sq_ft', 'units_in_building', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'zip_29401', 'zip_29403', 'zip_29406', 'zip_29407', 'zip_29410', 'zip_29412', 'zip_29414', 'zip_29418', 'zip_29420', 'zip_29455', 'zip_29464', 'zip_29466', 'zip_29485', 'zip_29492', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'apartment', 'condo', 'house', 'townhouse', 'undefined']


In [214]:
normalizer = Normalization(axis=-1)
normalizer.adapt(X_train_feats)

In [234]:


model = tf.keras.models.Sequential([
    normalizer,
    Dense(10, activation='relu', name='L1', kernel_regularizer=tf.keras.regularizers.L2(0.04)),
    Dense(5, activation='relu', name='L3', kernel_regularizer=tf.keras.regularizers.L2(0.04)),
    Dense(1, activation='linear', name='L4', kernel_regularizer=tf.keras.regularizers.L2(0.02))
], name = 'FirstModel')

model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(0.005))


In [235]:
history = model.fit(X_train_feats, y_train, epochs=400, verbose=0)


In [236]:
results_train = model.evaluate(X_train_feats, y_train)
results_val = model.evaluate(X_val[cols], y_val)
results_test = model.evaluate(X_test[cols], y_test)
print("training results: ", results_train)
print("validation results: ", results_val)
print("test results: ", results_test)



14/14 [==============================] - 0s 965us/step - loss: 280.4908
training results:  201.83213806152344
validation results:  331.1092834472656
test results:  280.4908142089844


In [237]:
y_val_hat = model.predict(X_val[cols])
y_diff = y_val_had = y_val

Y = pd.DataFrame(data=y_diff)

14/14 [==============================] - 0s 800us/step


In [252]:
def get_diff_df():
    D = X_val
    D = D.merge(addr_val, on='address')
    D['y'] = y_val 
    D['y_hat'] = y_val_hat 
    D['diff'] = D.y - D.y_hat 
    D.sort_values(by='diff', inplace=True)
    D.to_csv('/tmp/D.csv', sep=';')
    return D

import time

def scroll_through_overestimates():
    D = get_diff_df()
    i = 0
    while True:
            row = D.iloc[i]
            print(row.address, row.url, "beds and baths: ", row.beds, row.baths, "price: ", row.y, "estimated: ", row.y_hat)
            i += 1
            time.sleep(10)


def scroll_through_underestimates():
    D = get_diff_df()
    i = D.shape[0] - 1
    while True:
            row = D.iloc[i]
            print(row.address, row.url, "beds and baths: ", row.beds, row.baths, "price: ", row.y, "estimated: ", row.y_hat)
            i -= 1
            time.sleep(10)


In [253]:
scroll_through_underestimates()

515 Robert Daniel Dr, Charleston, SC 29492  https://www.apartments.com/wharf-7-daniel-island-sc/p9jy1sv/ beds and baths:  1 1.0 price:  7000 estimated:  4411.5576
7737 Eagle Lake Rd, North Charleston, SC 29418  https://www.apartments.com/3-br-2-bath-house-7737-eagle-lake-rd-north-charleston-sc/0t2xc22/ beds and baths:  3 2.0 price:  4179 estimated:  2327.7185
2410 Alston Ave, Charleston, SC 29406  https://www.apartments.com/alston-arms-charleston-sc/1xceh14/ beds and baths:  2 1.0 price:  6000 estimated:  4282.914
2410 Alston Ave, Charleston, SC 29406  https://www.apartments.com/alston-arms-charleston-sc/1xceh14/ beds and baths:  2 1.0 price:  6000 estimated:  4282.914
1000 Channel Marker Way, Charleston, SC 29410  https://www.apartments.com/channel-family-apartment-homes-charleston-sc/5bylrtp/ beds and baths:  1 1.0 price:  4500 estimated:  3100.1484
1000 Channel Marker Way, Charleston, SC 29410  https://www.apartments.com/channel-family-apartment-homes-charleston-sc/5bylrtp/ beds and

KeyboardInterrupt: 